# Exploratory Data Analysis

Hypothesis: We can use the https://arxiv.org/pdf/2406.04313 method for increasing honesty

In [1]:
# autoreload your package
%load_ext autoreload
%autoreload 2
import adapter_overseer


In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [3]:
import warnings
# warnings.simplefilter("ignore")
# warnings.filterwarnings("ignore", ".*does not have many workers.*")
# warnings.filterwarnings("ignore", ".*divide by zero.*")

## numeric, plotting
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (7.0, 4)

## utils
from pathlib import Path
from tqdm.auto import tqdm
import logging, os, re
import collections, functools, itertools
from loguru import logger

from typing import List, Callable, Tuple, Dict, Optional
from jaxtyping import Float, Int
from torch import Tensor

# torch
# import pytorch_lightning as pl
from einops import rearrange, repeat, reduce
import torch
import torch.nn as nn


from baukit.nethook import get_module
from baukit import TraceDict

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

In [4]:
from adapter_overseer.config import ExtractConfig

cfg = ExtractConfig(max_length=300, num_shots=1)
cfg

ExtractConfig(datasets=('amazon_polarity',), datasets_ood='imdb', model='failspy/Llama-3-8B-Instruct-abliterated', collection_layers=('base_model.model.model.layers.10', 'base_model.model.model.layers.20'), batch_size=2, prompt_format=None, num_shots=1, max_length=300, max_examples=1000, seed=42, max_epochs=1)

## Load

In [5]:

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [6]:
# https://huggingface.co/blog/mlabonne/orpo-llama-3
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install -qqq flash-attn
    attn_implementation = "flash_attention_2"
    torch_dtype = torch.bfloat16
else:
    attn_implementation = "eager"
    torch_dtype = torch.float16
torch_dtype, device

(torch.bfloat16, device(type='cuda', index=0))

In [7]:
# load model
# quantization_config = BitsAndBytesConfig(load_in_8bit=True)
quantization_config = BitsAndBytesConfig(load_in_4bit=True,     bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch_dtype, bnb_4bit_use_double_quant=True,)
model = AutoModelForCausalLM.from_pretrained(cfg.model, device_map="auto", quantization_config=quantization_config,)
model

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): Ll

In [8]:
tokenizer = AutoTokenizer.from_pretrained(cfg.model)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = 'left'
tokenizer.truncation_side = 'left'

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
# https://old.reddit.com/r/LocalLLaMA/comments/1coizjy/tokenizer_config_of_llama3_changed_by_meta_in_hf/
tokenizer.eos_token # it's good

'<|eot_id|>'

In [10]:
# from peft import get_peft_config, get_peft_model, LoraConfig, TaskType
# \peft_config = LoraConfig(
#     task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1
# )
# model = get_peft_model(model, peft_config)


In [11]:
# from peft import prepare_model_for_int8_training
# # we need to apply some post-processing on the 8-bit model to enable training, let's freeze all our layers, and cast the layer-norm in float32 for stability. We also cast the output of the last layer in float32 for the same reasons.
# model = prepare_model_for_int8_training(model, output_embedding_layer_name="proj_out")

In [12]:
from peft import LoraConfig, PeftModel, LoraModel, LoraConfig, get_peft_model, IA3Config
# https://github.com/huggingface/peft/blob/main/src/peft/utils/constants.py
config = LoraConfig(
                        #r=32,
                         lora_alpha=10,  # from paper
                    # target_modules=["q_proj", "v_proj"], lora_dropout=0.05, bias="none"
                    )
config = IA3Config(
)

from peft import prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(model)

model = get_peft_model(model, config)
model.print_trainable_parameters()

# check adapter disabling works
with model.disable_adapter():
    model.print_trainable_parameters()

trainable params: 524,288 || all params: 8,030,785,536 || trainable%: 0.0065
trainable params: 0 || all params: 8,030,785,536 || trainable%: 0.0000


## Get data

In [13]:
# perhaps use load_preproc_datasets from sdb_probes_are_lie_detectors repo... /media/wassname/SGIronWolf/projects5/elk/sgd_probes_are_lie_detectors/src/prompts/prompt_loading.py

In [14]:
# load a dataset of paired prompts, to try and get the model to lie
from adapter_overseer.prompts.prompt_loading import load_preproc_datasets

N = cfg.max_examples
ds_tokens = load_preproc_datasets(
    cfg.datasets,
    tokenizer,
    N=N,
    seed=cfg.seed,
    num_shots=cfg.num_shots,
    max_length=cfg.max_length,
    prompt_format=cfg.prompt_format,
)
ds_tokens


tokenize:   0%|          | 0/3004 [00:00<?, ? examples/s]

truncated:   0%|          | 0/3004 [00:00<?, ? examples/s]

truncated:   0%|          | 0/3004 [00:00<?, ? examples/s]

prompt_truncated:   0%|          | 0/3004 [00:00<?, ? examples/s]

choice_ids:   0%|          | 0/3004 [00:00<?, ? examples/s]

...

Filter:   0%|          | 0/3004 [00:00<?, ? examples/s]

/media/wassname/SGIronWolf/projects5/elk/adapters_can_monitor_lies/.venv/lib/python3.10/site-packages/sklearn/model_selection/_split.py:2328: UserWarning: The groups parameter is ignored by StratifiedShuffleSplit
  warnings.warn(


Filter:   0%|          | 0/2177 [00:00<?, ? examples/s]

2024-06-11 06:46:03.588 | INFO     | adapter_overseer.prompts.prompt_loading:load_preproc_dataset:405 - num_rows (after filtering out truncated rows) 3004=>2177


Dataset({
    features: ['ds_string', 'example_i', 'answer', 'messages', 'answer_choices', 'template_name', 'label_true', 'label_instructed', 'instructed_to_lie', 'sys_instr_name', 'question', 'input_ids', 'attention_mask', 'truncated', 'length', 'prompt_truncated', 'choice_ids'],
    num_rows: 1001
})

## Train: transformers

https://github.com/huggingface/peft/blob/main/examples/int8_training/Finetune_opt_bnb_peft.ipynb

In [15]:
def coeffecient(t, T, alpha=1):
    return alpha * t / (2 * T), alpha * (1- t/(2*T))

T = 150
for t in range(1, T, 10):
    cs = coeffecient(t, T)
    print(f"{t}/{T} {cs[0]:2.2f} {cs[1]:2.2f}")

1/150 0.00 1.00
11/150 0.04 0.96
21/150 0.07 0.93
31/150 0.10 0.90
41/150 0.14 0.86
51/150 0.17 0.83
61/150 0.20 0.80
71/150 0.24 0.76
81/150 0.27 0.73
91/150 0.30 0.70
101/150 0.34 0.66
111/150 0.37 0.63
121/150 0.40 0.60
131/150 0.44 0.56
141/150 0.47 0.53


In [ ]:

# TODO make sure that multiple cols get passed into trainer
ds = ds_tokens.select_columns(['label_true', 'label_instructed' ,'instructed_to_lie', 'input_ids', 'attention_mask', 'choice_ids'])

In [ ]:
# TODO change the loss function!
# we need to modify the forward pass, so that it returns a different loss function
# but to calculate this we will need to residuals now, and as they werre
# loss_bad = mse(repr_current, repr_target)

# from transformers import SFTTrainer
from trl.trainer import SFTTrainer, SFTConfig
import torch.nn.functional as F
import transformers
from adapter_overseer.helpers.torch_helpers import clear_mem, switch
from adapter_overseer.helpers.scores import select_choices

In [ ]:
def collect_hs(hidden_states, collection_layers):
    """The residual stream is the diff of the hs."""
    # 8, l=33, b=2, input=500, h=4096
    # Tuple (one element for each generated token) of tuples (one element for each layer of the decoder) of torch.FloatTensor of shape (batch_size, generated_length, hidden_size).        
    # from forward
    residuals = rearrange(list(hidden_states), 'l b t h -> l b t h').diff(0)[collection_layers]
    return residuals

def generate_fwd(model, toks: Int[Tensor, "batch_size seq_len"], max_tokens_generated: int = 6,):

    all_toks = torch.zeros(
        (toks.shape[0], toks.shape[1] + max_tokens_generated),
        dtype=torch.long,
        device=toks.device,
    )
    all_toks[:, : toks.shape[1]] = toks

    for i in range(max_tokens_generated):
        out = model(all_toks[:, : -max_tokens_generated + i], return_dict=True)
        if i==0:
            first_end_logits = out["logits"][:, -1]
        next_tokens = out.logits[:, -1, :].argmax(dim=-1)
        all_toks[:, -max_tokens_generated + i] = next_tokens
    
    # get last
    last_residuals = collect_hs(out.hidden_states, cfg.collection_layers)
    return first_end_logits, last_residuals


In [18]:
class CustomSFTTrainer(SFTTrainer):
    """
    Custom SFTTrainer that orthoganalizes the repr of bad examples, and retains good repr of examples

    See: https://arxiv.org/pdf/2406.04313

    args:
        collection_layers: list of baukit layer names to collect
    """
    def __init__(self, *args, collection_layers: list, alpha=0.1, **kwargs):
        super(CustomSFTTrainer, self).__init__(*args, **kwargs)
        self.collection_layers = collection_layers
        self.alpha = alpha
        self.total_steps = self.args.max_steps

    def compute_loss(self, model, inputs, return_outputs=False):       

        batch = {'input_ids': inputs['input_ids'], 'attention_mask': inputs['attention_mask']}

        # # collect the residuals of the model
        # with torch.no_grad():
        #     with model.disable_adapter():
        #         orig_outputs = model(**batch, output_hidden_states=True)
        # outputs = model(**batch, output_hidden_states=True)

        # collect the residuals of the model
        _, last_residuals = generate_fwd(model, inputs['input_ids'])
        with torch.no_grad():
            with model.disable_adapter():
                first_end_logits_base, last_residuals_base = generate_fwd(model, inputs['input_ids'])

        # new_tokens = 6
        # gen_kargs = dict(output_hidden_states=True, return_dict_in_generate=True, use_cache=False, do_sample=False, min_new_tokens=new_tokens, max_new_tokens=new_tokens, output_logits=True)
        # # FIXME... this has no grad
        # # we need to chain forward. Taking probs from first token, hidden states from last/all
        # with torch.no_grad():
        #     with model.disable_adapter():
        #         orig_outputs = model.generate(**batch, **gen_kargs)
        # outputs = model.generate(**batch, **gen_kargs)


        # def collect_hs(hidden_states):
        #     """The residual stream is the diff of the hs."""
        #     # 8, l=33, b=2, input=500, h=4096
        #     # Tuple (one element for each generated token) of tuples (one element for each layer of the decoder) of torch.FloatTensor of shape (batch_size, generated_length, hidden_size).

        #     # take last generation of model.generate
        #     residuals = rearrange(list(hidden_states[-1]), 'l b t h -> l b t h').diff(0) # diff over layers
        #     # collect chosen layers
        #     return residuals[self.collection_layers]
        
        #     # from forward
        #     residuals = rearrange(list(hidden_states), 't l b h -> l b t h').diff(0)[self.collection_layers]
        #     return residuals

        rep_adapt = last_residuals
        rep_orig = last_residuals_base
        # "for enhanced robustness, we apply the short circuit loss to both the user and assistant text within the short circuit set for large language models and agents."

        # so now we have a mixed batch of good and bad outputs
        # get probs of each choice
        # compare to labels to seperate into good and bad
        choice_ids = inputs['choice_ids'].detach().cpu().long()
        # label_instructed = inputs['label_true'] ^ inputs['instructed_to_lie']
        label_true = inputs['label_true']

        # does the underlying model get it right or wrong?
        # (generations, batch, tokens), gen first generation, last token
        end_logits = first_end_logits_base
        probs = torch.softmax(end_logits, -1)
        choice_probs = select_choices(probs, choice_ids).sum(2)
        binary_ans = choice_probs[:, 1] / (choice_probs.sum(1) + 1e-12)
        correct_truth_telling = switch(binary_ans, label_true)
        # correct_instruction_following = switch(binary_ans, label_instructed)

        mask_desired = correct_truth_telling>0.5


        # get coeffecient
        steps = self.state.global_step + 1
        c_s, c_r = coeffecient(steps, self.total_steps)
        c_s = torch.tensor(c_s).to(rep_orig.dtype)
        c_r = torch.tensor(c_r).to(rep_orig.dtype)

        loss_retain = F.mse_loss(rep_orig, rep_adapt, reduction='none' )[mask_desired]
        if loss_retain.numel() == 0:
            loss_retain = 0
        else:
            loss_retain = loss_retain.mean()
        loss_rr = F.relu(F.cosine_similarity(rep_orig, rep_adapt, dim=1))[~mask_desired]
        if loss_rr.numel() == 0:
            loss_rr = 0
        else:
            loss_rr = loss_rr.mean()
        loss = loss_rr * c_s + c_r * loss_retain
        loss = loss
        if steps % 20 == 0:
            logger.debug(f"steps: {steps}, c_r: {c_r}, loss_rr: {loss_rr:2.3f}, loss_retain: {loss_retain:2.3f}, loss={loss:2.3f}, mask_desired: {(mask_desired*1.0).mean():2.3f}")
        
        return (loss, outputs) if return_outputs else loss
    



clear_mem()

# see https://github.com/huggingface/trl/blob/main/trl/trainer/sft_trainer.py#L58
trainer = CustomSFTTrainer(
    model=model,
    train_dataset=ds,
    collection_layers=[10, 20],
    # max_seq_length=cfg.max_length,
    args=SFTConfig(
        # see https://github.com/huggingface/trl/blob/main/trl/trainer/sft_config.py#L21
        max_seq_length=cfg.max_length,
        per_device_train_batch_size=2, # 18GB/24GB
        gradient_accumulation_steps=8, # we want to accumulate the gradients to make the batch size larger, so we have sufficient examples of good and bad behaviour to learn from
        warmup_steps=10,
        max_steps=150, # 150 steps of batch=16 in paper
        learning_rate=3e-4, # from paper
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        remove_unused_columns=False,
    ),
    # data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
) 
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

clear_mem()
trainer.train()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
max_steps is given, it will override any value given in num_train_epochs


  0%|          | 0/150 [00:00<?, ?it/s]

/media/wassname/SGIronWolf/projects5/elk/adapters_can_monitor_lies/.venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/media/wassname/SGIronWolf/projects5/elk/adapters_can_monitor_lies/.venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
/media/wassname/SGIronWolf/projects5/elk/adapters_can_monitor_lies/.venv/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter s

AssertionError: No inf checks were recorded for this optimizer.

In [ ]:
# model = None
# clear_mem()

In [ ]:
# save
model.save_pretrained("../outputs/hs_adapter")

# Eval

In [ ]:
from datasets import load_dataset
# multiple_choice
from torch.utils.data import DataLoader
# dataset = load_dataset("truthfulqa/truthful_qa", "multiple_choice")

# HACK it was stalling for hours, so I loaded it locally
dataset = load_dataset("../data/truthful_qa")['validation']
dataset

In [ ]:
model.config.use_cache = True

In [ ]:
# https://github.dev/likenneth/honest_llama/blob/b92beb28deccd7ec6b26de7ebf9920122cfd15cd/utils.py#L72
# print(row)

def format_prompt(row):
    prompt = f"The is a multiple choice question. Question: {row['question']}\n"
    for i, choice in enumerate(row['mc2_targets']['choices']):
        prompt += f"{i+1}. {choice}\n"
    prompt += f"\nThe correct answer is number "

    choices = [str(i) for i in range(len(row['mc2_targets']['labels']))]
    return {'text': prompt, 
            'label': [np.argmax(row['mc2_targets']['labels'])],
            'choices': choices,
            'num_choices': len(choices),
            }

dataset1 = dataset.map(format_prompt)

In [ ]:
max([len(r['labels']) for r in dataset['mc2_targets']])

In [ ]:
# get our choice ids
choices = [str(i) for i in range(13)]
choice_ids = [tokenizer(c, add_special_tokens=False).input_ids[0] for c in choices]
choice_ids

In [ ]:

def tokenization(example):
    o = tokenizer(example["text"], padding="max_length", truncation=True, max_length=cfg.max_length, return_tensors="pt")
    return o

dataset2 = dataset1.map(tokenization, batched=True).select_columns([ 'label', 'input_ids', 'attention_mask', 
                                                                    'num_choices'
                                                                    ]).with_format("torch")
dataset2

# Generate

In [ ]:
i = 1
row = dataset1[i]
# row

In [ ]:
max_new_tokens = 32
s = row['text']
inputs = tokenizer(s, truncation=True, max_length=cfg.max_length, return_tensors="pt").to(device)
model.eval()
with torch.no_grad():
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens, min_new_tokens=max_new_tokens, do_sample=False)
    out_s = tokenizer.batch_decode(outputs, skip_special_tokens=False)
f"`{out_s[0]}`"

In [ ]:
with torch.no_grad():
    with model.disable_adapter():
        out2 = model.generate(**inputs, max_new_tokens=max_new_tokens, min_new_tokens=max_new_tokens, do_sample=False)
out_s2 = tokenizer.batch_decode(out2, skip_special_tokens=False)
f"`{out_s2[0]}`"

### Eval cls

In [ ]:
# https://github.dev/sylinrl/TruthfulQA/blob/fdd8ad1c0d00a478cf8b0bb41a3ad8378c16293b/truthfulqa/models.py#L311


probs = []
base_probs = []

dl = DataLoader(
    dataset2, batch_size=4, num_workers=0)
for b in tqdm(dl):
    inputs = {'input_ids': b['input_ids'], 'attention_mask': b['attention_mask']}
    with torch.no_grad():
        with model.disable_adapter():
            out_base = model(**inputs)
        out = model(**inputs)

        for j in range(len(out["logits"])):
            n = b['num_choices'][j]
            b_choice_ids = choice_ids[:n]
            label = b['label'][j, 0]

            choice_probs_base = out_base["logits"][j, -1, b_choice_ids].softmax(dim=-1)
            choice_probs = out["logits"][j, -1, b_choice_ids].softmax(dim=-1)
            prob = choice_probs[label].item()
            prob_base = choice_probs_base[label].item()
            probs.append(prob)
            base_probs.append(prob_base)
        

In [ ]:
choice_probs_base-choice_probs

In [ ]:
acc = ((torch.tensor(probs)>0.5)*1.0).mean()
base_acc = ((torch.tensor(base_probs)>0.5)*1.0).mean()
acc, base_acc

In [ ]:
prob_correct = torch.tensor(probs).mean()
prob_base_correct = torch.tensor(base_probs).mean()
prob_correct, prob_base_correct